# Llama3 Cookbook with Groq

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/llama3_cookbook_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meta developed and released the Meta [Llama 3](https://ai.meta.com/blog/meta-llama-3/) family of large language models (LLMs), a collection of pretrained and instruction tuned generative text models in 8 and 70B sizes. The Llama 3 instruction tuned models are optimized for dialogue use cases and outperform many of the available open source chat models on common industry benchmarks.

In this notebook, we demonstrate how to use Llama3 with LlamaIndex for a comprehensive set of use cases. 
1. Basic completion / chat 
2. Basic RAG (Vector Search, Summarization)
3. Advanced RAG (Routing)
4. Text-to-SQL 
5. Structured Data Extraction
6. Chat Engine + Memory
7. Agents


We use Llama3-8B and Llama3-70B through Groq.

## Installation and Setup

In [ ]:
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface
!pip install llama-parse

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Setup LLM using Groq

To use Groq, you need to make sure that `GROQ_API_KEY` is specified as an environment variable.

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "<GROQ_API_KEY>"

In [ ]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192")
llm_70b = Groq(model="llama3-70b-8192")

### Setup Embedding Model

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Define Global Settings Configuration

In LlamaIndex, you can define global settings so you don't have to pass the LLM / embedding model objects everywhere.

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

Here you'll download data that's used in section 2 and onwards.

We'll download some articles on Kendrick, Drake, and their beef (as of May 2024).

In [ ]:
!mkdir data
!wget "https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1" -O data/drake_kendrick_beef.pdf
!wget "https://www.dropbox.com/scl/fi/nts3n64s6kymner2jppd6/drake.pdf?rlkey=hksirpqwzlzqoejn55zemk6ld&st=mohyfyh4&dl=1" -O data/drake.pdf
!wget "https://www.dropbox.com/scl/fi/8ax2vnoebhmy44bes2n1d/kendrick.pdf?rlkey=fhxvn94t5amdqcv9vshifd3hj&st=dxdtytn6&dl=1" -O data/kendrick.pdf

### Load Data

We load data using LlamaParse by default, but you can also choose to opt for our free pypdf reader (in SimpleDirectoryReader by default) if you don't have an account! 

1. LlamaParse: Signup for an account here: cloud.llamaindex.ai. You get 1k free pages a day, and paid plan is 7k free pages + 0.3c per additional page. LlamaParse is a good option if you want to parse complex documents, like PDFs with charts, tables, and more. 

2. Default PDF Parser (In `SimpleDirectoryReader`). If you don't want to signup for an account / use a PDF service, just use the default PyPDF reader bundled in our file loader. It's a good choice for getting started!

In [ ]:
from llama_parse import LlamaParse

docs_kendrick = LlamaParse(result_type="text").load_data("./data/kendrick.pdf")
docs_drake = LlamaParse(result_type="text").load_data("./data/drake.pdf")
docs_both = LlamaParse(result_type="text").load_data(
    "./data/drake_kendrick_beef.pdf"
)


# from llama_index.core import SimpleDirectoryReader

# docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
# docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()
# docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

Started parsing the file under job_id 391f5fe8-aed3-46a3-af7d-18341b1b20d7
Started parsing the file under job_id 08c335d5-417b-4249-b53d-a7a9b65293a8
Started parsing the file under job_id e3a91a73-5db0-4df0-9590-c9393cb048cf


## 1. Basic Completion and Chat

### Call complete with a prompt

In [ ]:
response = llm.complete("do you like drake or kendrick better?")

print(response)

I'm just an AI, I don't have personal preferences or opinions, nor do I have the capacity to enjoy or dislike music. I can provide information and insights about different artists and their work, but I don't have personal feelings or biases.

However, I can tell you that both Drake and Kendrick Lamar are highly acclaimed and influential artists in the music industry. They have both received widespread critical acclaim and have won numerous awards for their work.

Drake is known for his introspective and emotive lyrics, as well as his ability to blend different genres such as hip-hop, R&B, and pop. He has been praised for his storytelling ability and his ability to connect with his audience.

Kendrick Lamar, on the other hand, is known for his socially conscious lyrics and his ability to tackle complex issues such as racism, inequality, and social justice. He has been praised for his lyrical depth and his ability to blend different genres such as hip-hop, jazz, and funk.

Ultimately, wh

In [ ]:
stream_response = llm.stream_complete(
    "you're a drake fan. tell me why you like drake more than kendrick"
)

for t in stream_response:
    print(t.delta, end="")

Man, I'm a die-hard Drake fan, and I gotta say, I love the 6 God for many reasons. Now, I know some people might say Kendrick is the king of hip-hop, and I respect that, but for me, Drake's got something special. Here's why:

1. **Relatability**: Drake's lyrics are like a diary entry. He's got this ability to tap into the emotions and struggles of everyday people. His songs are like a reflection of our own experiences, you know? He's not just rapping about gangsta life or material possessions; he's talking about the real stuff, like relationships, fame, and the struggles of growing up. That's what makes his music so relatable and authentic.

2. **Vocal delivery**: Drake's got this smooth, melodic flow that's unmatched. His vocals are like butter – they just glide over the beat. He's got this effortless swag that makes his songs feel like a warm hug on a cold day. Kendrick's got a great flow too, but Drake's got this unique, laid-back vibe that's hard to replicate.

3. **Storytelling**:

### Call chat with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Kendrick."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [ ]:
print(response)

assistant: "I'm the king of the game, no debate
My rhymes so tight, they're like a weight
I'm the voice of the streets, the people's champ
My flow's on fire, leaving the haters in the slam"


## 2. Basic RAG (Vector Search, Summarization)

### Basic RAG (Vector Search)

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
response = query_engine.query("Tell me about family matters")

In [ ]:
print(str(response))

The concept of "Family Matters" is a central theme in the ongoing beef between Drake and Kendrick Lamar. It refers to a seven-and-a-half-minute diss track released by Drake in response to Kendrick's diss track "Family Matters." The track is a scathing attack on Kendrick, with Drake addressing various allegations and accusations made by Kendrick. The track is notable for its dark and sinister tone, with Drake delivering a series of personal attacks on Kendrick and his family. The track also features Drake addressing his own family, including his son Adonis and his parents, Dennis and Sandi Graham.


### Basic RAG (Summarization)

In [ ]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(docs_both)
summary_engine = summary_index.as_query_engine()

In [ ]:
response = summary_engine.query(
    "Given your assessment of this article, who won the beef?"
)

In [ ]:
print(str(response))

It's difficult to declare a clear winner in this beef, as both Kendrick Lamar and Drake have delivered scathing diss tracks, and the beef has been marked by a series of intense exchanges.


## 3. Advanced RAG (Routing)

### Build a Router that can choose whether to do vector search or summarization

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, verbose=True, llm=llm_70b
)

response = query_engine.query(
    "Tell me about the song meet the grahams - why is it significant"
)

Selecting query engine 0: The question asks for specific facts about the song 'Meet the Grahams', so a search for specific facts is required..


In [ ]:
print(response)

The "Meet the Grahams" artwork is significant because it's the full picture that Kendrick Lamar teased earlier on "6.16 in LA." It shows a pair of Maybach gloves, a shirt, receipts, and prescription bottles, including one for Ozempic prescribed to Drake.


## 4. Text-to-SQL 

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
!wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook.zip"
!unzip "./data/chinook.zip"

--2024-05-10 23:40:37--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 2606:4700:3037::6815:1e8d, 2606:4700:3037::ac43:acfa, 104.21.30.141, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|2606:4700:3037::6815:1e8d|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘./data/chinook.zip’

./data/chinook.zip  100%[===================>] 298.43K  --.-KB/s    in 0.02s   

2024-05-10 23:40:37 (13.9 MB/s) - ‘./data/chinook.zip’ saved [305596/305596]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  ./data/chinook.zip
  inflating: chinook.db              


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    llm=llm_70b,
)

In [ ]:
response = query_engine.query("What are some albums?")

print(response)

Here are some albums: For Those About To Rock We Salute You, Balls to the Wall, Restless and Wild, Let There Be Rock, Big Ones, Jagged Little Pill, Facelift, Warner 25 Anos, Plays Metallica By Four Cellos, and Audioslave.


In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

print(response)

Here are 5 artists: AC/DC, Accept, Aerosmith, Alanis Morissette, and Alice In Chains.


This last query should be a more complex join

In [ ]:
response = query_engine.query(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

print(response)

Here are three tracks from the legendary Australian rock band AC/DC: "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".


In [ ]:
print(response.metadata["sql_query"])

SELECT tracks.Name FROM tracks INNER JOIN albums ON tracks.AlbumId = albums.AlbumId INNER JOIN artists ON albums.ArtistId = artists.ArtistId WHERE artists.Name = 'AC/DC' LIMIT 3;


## 5. Structured Data Extraction

An important use case for function calling is extracting structured objects. LlamaIndex provides an intuitive interface for this through `structured_predict` - simply define the target Pydantic class (can be nested), and given a prompt, we extract out the desired object.

**NOTE**: Since there's no native function calling support with Llama3, the structured extraction is performed by prompting the LLM + output parsing.

In [ ]:
from llama_index.llms.groq import Groq
from llama_index.core.prompts import PromptTemplate
from pydantic import BaseModel


class Restaurant(BaseModel):
    """A restaurant with name, city, and cuisine."""

    name: str
    city: str
    cuisine: str


llm = Groq(model="llama3-8b-8192", pydantic_program_mode="llm")
prompt_tmpl = PromptTemplate(
    "Generate a restaurant in a given city {city_name}"
)

In [ ]:
restaurant_obj = llm.structured_predict(
    Restaurant, prompt_tmpl, city_name="Miami"
)
print(restaurant_obj)

name='Café Havana' city='Miami' cuisine='Cuban'


## 6. Adding Chat History to RAG (Chat Engine)

In this section we create a stateful chatbot from a RAG pipeline, with our chat engine abstraction.

Unlike a stateless query engine, the chat engine maintains conversation history (through a memory module like buffer memory). It performs retrieval given a condensed question, and feeds the condensed question + context + chat history into the final LLM prompt.

Related resource: https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_condense_plus_context/

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about the Kendrick and Drake beef."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [ ]:
response = chat_engine.chat(
    "Tell me about the songs Drake released in the beef."
)
print(str(response))

Condensed question: Tell me about the songs Drake released in the beef.
Context: https://www.gq.com/story/the-kendrick-lamar-drake-beef-explained                                        27/34
---
     5/10/24, 10:08 PM                                        The Kendrick Lamar/Drake Beef, Explained | GQ
          May 5: Drake hits back with “The Heart Part 6”
                     The HEART PART 6   DRAKE


THE HEART PART 6 - DRAKETHE HEART PART 6 - DRAKE
          The most productive weekend in the history of rap beef continues, with Drake saying
          fuck all to the Sunday Truce and doing exactly what Joe Budden advised: to hit back at
          Kendrick's onslaught with a record in the vein of his time-stamp series, straight bars over
          a hard beat. Only, Kendrick already beat him to a time-stamp title last week with “6:16
          in LA,” so Drake counters by co-opting one of Kendrick's recurring series: “The Heart.”
          (The last official entry, “The Heart Part 5,

In [ ]:
response = chat_engine.chat("What about Kendrick?")
print(str(response))

Condensed question: What did Kendrick Lamar release in response to Drake's "The Heart Part 6"?
Context: https://www.gq.com/story/the-kendrick-lamar-drake-beef-explained                                        27/34
---
     5/10/24, 10:08 PM                                        The Kendrick Lamar/Drake Beef, Explained | GQ
          May 5: Drake hits back with “The Heart Part 6”
                     The HEART PART 6   DRAKE


THE HEART PART 6 - DRAKETHE HEART PART 6 - DRAKE
          The most productive weekend in the history of rap beef continues, with Drake saying
          fuck all to the Sunday Truce and doing exactly what Joe Budden advised: to hit back at
          Kendrick's onslaught with a record in the vein of his time-stamp series, straight bars over
          a hard beat. Only, Kendrick already beat him to a time-stamp title last week with “6:16
          in LA,” so Drake counters by co-opting one of Kendrick's recurring series: “The Heart.”
          (The last official en

## 7. Agents

Here we build agents with Llama 3. We perform RAG over simple functions as well as the documents above.

### Agents And Tools

In [ ]:
import json
from typing import Sequence, List

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent

import nest_asyncio

nest_asyncio.apply()

### Define Tools

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer"""
    return a / b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
divide_tool = FunctionTool.from_defaults(fn=divide)

### ReAct Agent

In [ ]:
agent = ReActAgent.from_tools(
    [multiply_tool, add_tool, subtract_tool, divide_tool],
    llm=llm_70b,
    verbose=True,
)

### Querying

In [ ]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: Now I have the result of the addition, I need to multiply it by 5.
Action: multiply
Action Input: {'a': 123, 'b': 5}
Observation: 615
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 615
615


### ReAct Agent With RAG QueryEngine Tools

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

### Create ReAct Agent using RAG QueryEngine Tools

In [ ]:
drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_search",
        description="Useful for searching over Kendrick's life.",
    ),
)

query_engine_tools = [drake_tool, kendrick_tool]

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,  ## TODO: define query tools
    llm=llm_70b,
    verbose=True,
)

### Querying

In [ ]:
response = agent.chat("Tell me about how Kendrick and Drake grew up")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: kendrick_search
Action Input: {'input': "Kendrick Lamar's childhood"}
Observation: Kendrick Lamar was born on June 17, 1987, in Compton, California. He is the first child of Kenneth "Kenny" Duckworth, a former gang hustler, and Paula Oliver, a hairdresser. Both of his parents are African Americans from the South Side of Chicago. When they were teenagers, they relocated to Compton in 1984 due to his father's affiliation with the Gangster Disciples. Lamar was named after singer-songwriter Eddie Kendricks of the Temptations.
Thought: I need more information about Drake's childhood to compare their upbringings.
Action: drake_search
Action Input: {'input': "Drake's childhood"}
Observation: Drake's parents divorced when he was five years old. After the divorce, he and his mother remained in Toronto; his father returned to Memphis, where he was incarcerated for a number of years